#1. Data

1.1 Load

In [1]:
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np

df = pd.read_csv('smartphones.csv')
remove_n = 1700
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df = df.drop(drop_indices)
df = df.reset_index(drop=True)

df.loc[df['price_range'] == 3, 'price_range'] = 2
y = df['price_range']
print(df['price_range'].value_counts())

df = df.drop(columns=['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi', 'price_range'])
df = df.astype('float') 
# нормализация
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])
#df = (df-df.min())/(df.max()-df.min())
df.info()
classes = set(y)
print(df.head(5))
# 3 класса по дороговизне
classes

2    158
0     74
1     68
Name: price_range, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  300 non-null    float64
 1   clock_speed    300 non-null    float64
 2   fc             300 non-null    float64
 3   int_memory     300 non-null    float64
 4   m_dep          300 non-null    float64
 5   mobile_wt      300 non-null    float64
 6   n_cores        300 non-null    float64
 7   pc             300 non-null    float64
 8   px_height      300 non-null    float64
 9   px_width       300 non-null    float64
 10  ram            300 non-null    float64
 11  sc_h           300 non-null    float64
 12  sc_w           300 non-null    float64
 13  talk_time      300 non-null    float64
dtypes: float64(14)
memory usage: 32.9 KB
   battery_power  clock_speed        fc  int_memory     m_dep  mobile_wt  \
0       0.0749

{0, 1, 2}

1.2 Bootstrap

In [ ]:
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

1.3 Classification

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

models = [GaussianNB(), KNeighborsClassifier(), LinearSVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

0.92
0.8133333333333334
0.92


1.4 Aggregation of predictions

In [ ]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
0,NaN,2.0,NaN
1,1.0,1.0,1.0
2,NaN,0.0,NaN
3,NaN,2.0,NaN
4,2.0,NaN,NaN


In [ ]:
preds_df.to_csv('preds.csv')

#2. Ontology

2.1 Epistemic and doxastic logic

2.2 Creating ontology

In [ ]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2.3 Creating classes and properties

In [ ]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

2.4 Creating instances

In [ ]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto_with_instances.owl', format='ntriples')

2.5 Inferring relations

In [ ]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('onto_with_relations.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packa

* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_2
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_0
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_2
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_0
* Owlready * Adding relation onto.inst106_1 tsr onto.inst106_1
* Owlready * Adding relation onto.inst106_1 ts onto.inst106_1
* Owlready * Adding relation onto.inst219_2 tsr onto.inst219_2
* Owlready * Adding relation onto.inst219_2 tsr onto.inst219_0
* Owlready * Adding relation onto.inst219_2 ts onto.inst219_2
* Owlready * Adding relation onto.inst219_2 ts onto.inst219_0
* Owlready * Adding relation onto.inst246_2 tsr onto.inst246_2
* Owlready * Adding relation onto.inst176_1 tsr onto.inst176_1
* Owlready * Adding relation onto.inst176_1 ts onto.inst176_1
* Owlready * Adding relation onto.inst159_1 tsr onto.inst159_1
* Owlready * Adding relation onto.inst159_1 ts onto.inst159_1
* Owlready * Adding relation onto.inst217_1 tsr onto.inst217_1
* Owlre

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


2.6 OWA/CWA

In [ ]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

# onto.save('onto_cwa.owl', format='ntriples')

2.7 Inferring classification

In [ ]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packa